# Imports and Version Control

In [33]:
# !pip uninstall keras
# !pip install keras==2.13.0  # As you specified this version

In [34]:
!pip install tensorflow==2.8.0
!pip install gym
!pip install keras==2.1.0
!pip install keras-rl2
!pip install h5py
!pip install Pillow
!pip install gym[atari]
!pip install atari-py

  Using cached tensorflow-2.8.0-cp310-cp310-win_amd64.whl (438.0 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.1.0
    Uninstalling Keras-2.1.0:
      Successfully uninstalled Keras-2.1.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.13.0
    Uninstalling tensorflow-2.13.0:
      Successfully uninstalled tensorflow-2.13.0
  Rolling back uninstall of tensorflow
  Moving to c:\users\clayb\anaconda3\envs\deep_py\lib\site-packages\tensorflow-2.13.0.dist-info\
   from C:\Users\clayb\anaconda3\envs\deep_py\Lib\site-packages\~ensorflow-2.13.0.dist-info


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\clayb\\Anaconda3\\envs\\deep_py\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



  Using cached Keras-2.1.0-py2.py3-none-any.whl (302 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.1.0 which is incompatible.
tensorflow-intel 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.8.0 which is incompatible.


  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Obtaining dependency information for keras<2.14,>=2.13.1 from https://files.pythonhosted.org/packages/2e/f3/19da7511b45e80216cbbd9467137b2d28919c58ba1ccb971435cb631e470/keras-2.13.1-py3-none-any.whl.metadata
  Using cached keras-2.13.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Obtaining dependency information for tensorboard-data-server<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/da/61/6e9ff8258422d287eec718872fb71e05324356722ab658c8afda25f51539/tensorboard_data_server-0.7.1-py3-none-any.whl.metadata
  Using cached tensorboard_data_server-0.7.1-py3-none-any.whl.metadata (1.1 kB)
Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
Using cached tensorboard_data_server-0.7.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-serve

In [1]:
import numpy as np

import gym
import tensorflow as tf


# import atari

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [36]:
import gymnasium

# Train Model

In [37]:
# Initialize Gym environment
ENV_NAME = 'Breakout-v0'
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

(693650678, 2973253328)

In [38]:
# Number of actions
nb_actions = env.action_space.n

In [39]:
# Define the model architecture
input_shape = (84, 84)
window_length = 4




train_model = Sequential()
train_model.add(Flatten(input_shape=(3,) + env.observation_space.shape))
train_model.add(Dense(64, activation='relu'))
train_model.add(Dense(32, activation='relu'))
train_model.add(Dense(16, activation='relu'))
train_model.add(Dense(nb_actions, activation='linear'))


In [40]:
# Number of actions
nb_actions = env.action_space.n

In [41]:
# Define the model architecture
input_shape = (84, 84)
window_length = 4

model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

In [48]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [ ]:
!pip install baselines
from baselines.common.atari_wrappers import make_atari, wrap_deepmind
from baselines import deepq

In [ ]:
def main():
    env = make_atari('BreakoutNoFrameskip-v4')
    env = wrap_deepmind(env, frame_stack=True)
    
    model = deepq.models.cnn_to_mlp(
        convs=[(32, 8, 4), (64, 4, 2), (64, 3, 1)],
        hiddens=[256],
        dueling=True
    )
    
    act = deepq.learn(
        env,
        q_func=model,
        lr=0.00025,
        max_timesteps=2000000,
        buffer_size=10000,
        exploration_fraction=0.1,
        exploration_final_eps=0.01,
        print_freq=10,
        param_noise=True
    )
    
    act.save("breakout_model.pkl")

if __name__ == '__main__':
    main()

In [49]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000000, window_length=window_length)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])


TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

In [ ]:
# Train the agent
dqn.fit(env, nb_steps=1750000, visualize=False, verbose=2)

In [ ]:
# Save the final policy network
dqn.save_weights('policy.h5', overwrite=True)

# Play Model

In [ ]:
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

In [ ]:
# Configure and compile the agent
memory = SequentialMemory(limit=1000000, window_length=window_length)
policy = GreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50000,
               target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

In [ ]:
# Load the trained weights
dqn.load_weights('policy.h5')

In [ ]:
# Evaluate the agent
dqn.test(env, nb_episodes=10, visualize=True)